In [1]:
import os
import yaml
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import json

Preprocessing

In [2]:
# Load and preprocess data
def load_data(directory):
    data = []
    labels = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".yml"):
            with open(os.path.join(directory, filename), 'r') as yaml_file:
                hand_data = yaml.safe_load(yaml_file)
                landmarks = [point['x'] for point in hand_data['hand_landmarks']]
                data.append(landmarks)
                labels.append(directory.split("/")[-1])  # Assuming the class label is the last part of the directory

    return np.array(data), labels

In [3]:
# Prepare dataset
signs_dir = "Signs"
classes = os.listdir(signs_dir)
all_data = []
all_labels = []


In [4]:
# Convert list to dictionary
my_dict = {i: classes[i] for i in range(len(classes))}

# Save the dictionary to a JSON file
with open('labels_info.json', 'w') as json_file:
    json.dump(my_dict, json_file)

In [5]:
print(all_labels)
print(classes)

[]
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Hello', 'I', 'I love You', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [6]:
for sign_class in classes:
    data, labels = load_data(os.path.join(signs_dir, sign_class))
    all_data.extend(data)
    all_labels.extend(labels)


In [7]:
# Convert labels to integers
label_to_int = {label: i for i, label in enumerate(set(all_labels))}
all_labels = [label_to_int[label] for label in all_labels]

In [8]:
print(label_to_int)

{'Signs\\K': 0, 'Signs\\Hello': 1, 'Signs\\S': 2, 'Signs\\O': 3, 'Signs\\A': 4, 'Signs\\B': 5, 'Signs\\U': 6, 'Signs\\Z': 7, 'Signs\\V': 8, 'Signs\\N': 9, 'Signs\\Y': 10, 'Signs\\T': 11, 'Signs\\Q': 12, 'Signs\\F': 13, 'Signs\\I love You': 14, 'Signs\\D': 15, 'Signs\\G': 16, 'Signs\\M': 17, 'Signs\\P': 18, 'Signs\\W': 19, 'Signs\\H': 20, 'Signs\\X': 21, 'Signs\\L': 22, 'Signs\\J': 23, 'Signs\\E': 24, 'Signs\\I': 25, 'Signs\\C': 26, 'Signs\\R': 27}


Training

In [9]:
# Convert data to numpy array
X_data = np.array(all_data)
y_labels = np.array(all_labels)


In [10]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_data, y_labels, test_size=0.2, random_state=42)


In [11]:
# Build the model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(set(all_labels)), activation='softmax')
])

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
140/140 [==============================] - 1s 3ms/step - loss: 3.3151 - accuracy: 0.0413 - val_loss: 3.2993 - val_accuracy: 0.0696
Epoch 2/100
140/140 [==============================] - 0s 2ms/step - loss: 3.2267 - accuracy: 0.1089 - val_loss: 3.1522 - val_accuracy: 0.1205
Epoch 3/100
140/140 [==============================] - 0s 2ms/step - loss: 2.9657 - accuracy: 0.1513 - val_loss: 2.8206 - val_accuracy: 0.1321
Epoch 4/100
140/140 [==============================] - 0s 2ms/step - loss: 2.6010 - accuracy: 0.2480 - val_loss: 2.4688 - val_accuracy: 0.2527
Epoch 5/100
140/140 [==============================] - 0s 2ms/step - loss: 2.2621 - accuracy: 0.3723 - val_loss: 2.1783 - val_accuracy: 0.3589
Epoch 6/100
140/140 [==============================] - 0s 2ms/step - loss: 1.9711 - accuracy: 0.4853 - val_loss: 1.9076 - val_accuracy: 0.4750
Epoch 7/100
140/140 [==============================] - 0s 2ms/step - loss: 1.7281 - accuracy: 0.5647 - val_loss: 1.6646 - val_accuracy: 0.5920

In [15]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

35/35 [==============================] - 0s 1ms/step - loss: 0.1825 - accuracy: 0.9580
Test Accuracy: 0.9580357074737549


In [16]:
# Save the model and label mapping
model.save('sign_language_model.h5')

c:\Users\yash\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Create a dictionary mapping class labels to class names
label_to_class_name = {label_to_int[label]: label for label in label_to_int}

Testing Data

In [18]:
# Save the label mapping to a JSON file
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_to_class_name, json_file)

In [19]:
# Load the trained model
model = tf.keras.models.load_model('sign_language_model.h5')

In [20]:
# Load the label information from the JSON file
with open('label_mapping.json', 'r') as json_file:
    label_mapping = json.load(json_file)

In [21]:
# Path to your test landmarks YAML file
test_landmarks_path = 'Signs/Hello/left_10.yml'  # Replace with the path to your test landmarks YAML file

In [22]:
# Function to preprocess hand landmarks
def preprocess_hand_landmarks(hand_landmarks):
    landmarks = [point['x'] for point in hand_landmarks['hand_landmarks']]
    return np.array(landmarks)

In [23]:
# Load hand landmarks data from YAML file
with open(test_landmarks_path, 'r') as yaml_file:
    hand_landmarks_data = yaml.safe_load(yaml_file)

In [24]:
# Preprocess hand landmarks
hand_landmarks_processed = preprocess_hand_landmarks(hand_landmarks_data)


In [25]:
# Resize the data to match the input size expected by the model
hand_landmarks_resized = np.expand_dims(hand_landmarks_processed, axis=0)


In [26]:
# Make prediction
prediction = model.predict(hand_landmarks_resized)
predicted_class = np.argmax(prediction)
print(predicted_class)


1/1 [==============================] - 0s 84ms/step
22


In [27]:
# Map the predicted class to its corresponding class name
predicted_class_name = label_mapping.get(str(predicted_class), "Unknown")

# Extract only the last part of the path (class name)
predicted_class_name = os.path.basename(predicted_class_name)

In [28]:
# Display the predicted class name
print(f"Predicted Class Name: {predicted_class_name}")

Predicted Class Name: L
